In [ ]:
import logging
from pathlib import Path

import numpy as np
import rasterio
from rasterio.windows import Window, transform

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s %(levelname)s %(name)s %(message)s",
)

In [ ]:
total_tiles = 0
tile_size = 512
prefix = "IMG_"  # IMG to images, MASK to masks.
image_id = "S2A_MSIL2A_20160728T152642_N0211_R025_T18LTQ_20240316T000332"

In [ ]:
image = Path(f"data/{image_id}.tif")

output_folder = Path(f"results/generate_tiles/{image_id}")
output_folder.mkdir(parents=True, exist_ok=True)

with rasterio.open(image) as src:
    profile = src.profile
    profile.update(
        height=tile_size,
        width=tile_size,
        blockxsize=tile_size,
        blockysize=tile_size,
        tiled=True
    )

    num_tiles_x = int(np.ceil(src.width / tile_size))
    num_tiles_y = int(np.ceil(src.height / tile_size))

    logging.info(f"Number of tiles in x: {num_tiles_x}")
    logging.info(f"Number of tiles in y: {num_tiles_y}")
    logging.info(f"Total number of tiles: {num_tiles_x * num_tiles_y}")

    for j in range(num_tiles_x):
        for i in range(num_tiles_y):
            start_x = j * tile_size
            start_y = i * tile_size
            end_x = min(start_x + tile_size, src.width)
            end_y = min(start_y + tile_size, src.height)

            window = Window.from_slices((start_y, end_y), (start_x, end_x))

            tile = src.read(window=window)

            # If the tile is not the correct size, fill it with zeros.

            if tile.shape[1] != tile_size or tile.shape[2] != tile_size:
                padded_tile = np.zeros((tile.shape[0], tile_size, tile_size), dtype=tile.dtype)
                padded_tile[:, : tile.shape[1], : tile.shape[2]] = tile
                tile = padded_tile

            # Save the tile.

            tile_profile = profile.copy()
            tile_profile.update(
                height=tile_size,
                width=tile_size,
                transform=transform(window, src.transform)
            )

            tile_path = output_folder / f"{prefix}{total_tiles}.tif"
            with rasterio.open(tile_path, "w", **tile_profile) as dst:
                dst.write(tile)

            total_tiles += 1

    logging.info(f"Total number of tiles saved: {total_tiles}")